<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install tabula-py
!pip install pdfminer
!pip install PyPDF2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import io
import re
import os
import cv2
import tabula
import PyPDF2
import pytesseract
import pandas as pd
from PIL import Image
from io import StringIO
from google.colab import files
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [13]:
file_objects = files.upload()

Saving Historico - Posicao Consolidada XP - Cliente 04.pdf to Historico - Posicao Consolidada XP - Cliente 04 (1).pdf


In [14]:
# Converte o conteúdo do PDF para texto
def convert_pdf_to_txt(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text

#def remove_header_and_footer(text, cm, tm, fontDict, fontSize):
#    y = tm[5]
#    if y > 50 and y < 720:
#        parts.append(text)

def get_content_of_tables(pdf_file):
    # Extrair todas as tabelas do arquivo PDF
    tables = tabula.read_pdf(pdf_file, multiple_tables=True, pages="all")

    return tables


def get_text_from_page_images(reader):

    # Inicializa variáveis
    texts = []
    page_number = 0

    # Percore todas as páginas do documento
    for page in reader.pages:
        
        page_number += 1

        try:
            # Obtém os recursos da página
            xObject = page['/Resources']['/XObject'].get_object()

            image_number = 0
            # Percorre todos os objetos X da página
            for obj in xObject:

                image_number += 1

                # Verifica se o objeto é uma imagem
                if xObject[obj]['/Subtype'] == '/Image':
                    size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                    data = xObject[obj].get_data()
                    if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                        mode = "RGB"
                    else:
                        mode = "P"

                    # Cria uma imagem a partir dos dados
                    img = Image.frombytes(mode, size, data)
                    image_name = file_name[0:25] + "_" + str(page_number) + "_" + str(image_number) + ".png"
                    #image_path = pdf_path = "/content/gdrive/My Drive/Temp/" + image_name

                    # Salva a imagem
                    img.save(image_name)

                    # Carrega a imagem
                    image = cv2.imread(image_name)

                    # Converte a imagem para escala de cinza
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                    # Executa o OCR usando o Tesseract
                    text = pytesseract.image_to_string(image).strip()

                    # Adiciona o texto obtido ao array de textos
                    texts.append(text)
        except:
            texts.append(f'Não foi possível ler as imagens da página {page_number}')
    
    return texts

def create_new_df():

    # Cria um DataFrame utilizado no consolidado final
    return pd.DataFrame({
        'CLASSIFICAÇÃO': pd.Series(dtype='str'),
        'ATIVO': pd.Series(dtype='str'),
        'EXPOSIÇÃO': pd.Series(dtype='str'),
        'INSTITUIÇÃO': pd.Series(dtype='str'),
        'ATUAL': pd.Series(dtype='str'),
        'DATA': pd.Series(dtype='str'),
        'ON/OFF': pd.Series(dtype='str'),
        'DATA DE VENCIMENTO': pd.Series(dtype='str')
    })

def obtem_classificacao(ativo):

    # Verifica se é uma ativo no formato de Ticker
    founded = re.search(r'\b[A-Z]{4}(3|4|5|6|11|32|33|34|35)\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"
    
    # Define as palavras chave para cada Classificação
    dict_classificacoes = {
        "Ações": [
            "FIC FIA","FIA","FI Ações", "FI Açoes"
        ],
        "Renda Fixa": [
            "Renda Fixa","RF","CDB","LC","LF","LFSN","LFSC","LCA","LCI","CRA","CRI","Deb",
            "Debenture","Debênture","Tesouro","Pré","Pós","Pre","Pos","IPCA","IPCA+","LTN",
            "NTN","NTN-B","NTNB","DI","REF DI","CP","Credito Privado","Crédito Privado",
            "LP","Longo Prazo","Previdencia","Previdência","Prev","VGBL","PGBL"
        ],
        "Multimercado": [
            "FIC FIM","FIM","MM","Multimercado","Multi","COE","Multiestratégia",
            "Multiestratégia"
        ]
    }

    classes = []
    # Percorre cada uma das classificações
    for classificacao in dict_classificacoes:

        # Percorre as palavras-chave de cada uma das classificações
        for key in dict_classificacoes[classificacao]:

            # Verifica se a palavra-chave atual existe no nome do ativo
            founded = re.search(r'\b' + key + r'\b', ativo, flags=(re.IGNORECASE))
            if founded and not classificacao in classes:
                classes.append(classificacao)

    if len(classes) > 0:
        return ' / '.join(classes)

    # Se nenhuma classificação foi encontrada, verifica se é um ETF de investimento estrangeiro
    founded = re.search(r'\b[A-Z]{3}\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"

    return "***"

def get_pdf_texts():

    text_pypdf = ""
    text_pdfminer = ""

    for file in file_objects.keys():

        file_name, file_extension = os.path.splitext(file)

        if file_extension != '.pdf':
            continue

        # Lê o conteúdo do arquivo PDF
        text_pdfminer = convert_pdf_to_txt(file)
        
        reader = PyPDF2.PdfReader(file)
        page_number = 0

        for page in reader.pages:

            page_number += 1
            text_pypdf = text_pypdf + page.extract_text() #visitor_text=remove_header_and_footer)
        
        # print(get_text_from_page_images(reader))
    
    return text_pypdf, text_pdfminer

In [15]:
# Cria o DataFrame que conterá o consolidado final
df_result = create_new_df()

# Obtém os textos do pdf
text_pypdf, text_pdfminer = get_pdf_texts()

In [11]:
text_pypdf

" \nPREFIXADA\nPREFIXADA\nATIVO\nAPLICAÇÃO\nCARÊNCIA\nVENCIMENTO\nTAXA\nDISPONÍVEL\nGARANTIA\nVALOR \nAPLICADO\nPOSIÇÃO\nVALOR \nLÍQUIDO\nCDB PINE - NOV/2024\n09/11/2021\n08/11/2024\n08/11/2024\n+ 13,05%\n101\n0\nR$ 101.000,00\nR$ 112.744,61\nR$ 110.395,69\nCDB ORIGINAL - JUN/2025\n22/06/2021\n21/06/2025\n21/06/2025\n+ 10,60%\n216\n0\nR$ 216.000,00\nR$ 245.774,44\nR$ 240.563,91\n \nPÓS-FIXADA\nPÓS-FIXADA\nATIVO\nAPLICAÇÃO\nCARÊNCIA\nVENCIMENTO\nTAXA\nDISPONÍVEL\nGARANTIA\nVALOR \nAPLICADO\nPOSIÇÃO\nVALOR \nLÍQUIDO\nCDB SEMEAR - OUT/2025\n11/10/2021\n10/10/2025\n10/10/2025\n126,00% CDI\n63\n0\nR$ 63.000,00\nR$ 71.618,28\nR$ 69.894,62\n \nINFLAÇÃO\nINFLAÇÃO\nATIVO\nAPLICAÇÃO\nCARÊNCIA\nVENCIMENTO\nTAXA\nDISPONÍVEL\nGARANTIA\nVALOR \nAPLICADO\nPOSIÇÃO\nVALOR \nLÍQUIDO\nNTN-B \n- MAI/2055\n15/09/2021\n-\n15/05/2055\nIPC-A + 4,46%\n8\n0\nR$ 37.402,98\nR$ 40.569,67\nR$ 39.910,93\nNTN-B \n- AGO/2030\n11/10/2021\n-\n15/08/2030\nIPC-A + 4,53%\n23\n0\nR$ 94.178,49\nR$ 100.212,81\nR$ 98.848,06\nN

In [21]:
# Cria o dataFrame resultante
df_result = create_new_df()

# Regex para obter a Data de Emissão
regex_0 = "(?<=Data da Consulta: )\d{2}\/\d{2}\/\d{4}"
data_emissao = re.search(regex_0, text_pypdf, flags=(re.IGNORECASE)).group(0)

# Regex para obter todas as linhas que possuírem ativos de Renda Fixa
regex_1 = r'(?<=\n)(.+?)\s*-\s*[A-Z]{3}\/\d{4}\s*\d{2}\/\d{2}\/\d{4}\s*\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4})\s*[A-Z\-]*\s*[\+\-]\s*[\d,.]+%\s+\d+\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*(R\$\s*[\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}'
result_1 = re.findall(regex_1, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

# Regex para obter todas as linhas que possuírem ativos de Renda Fixa Pós-Fixados
regex_2 = r'(?<=\n)(.+?)\s*(\d{2}\/\d{2}\/\d{4})\s*[\d,.]+\s*[\d,.]+\s*R\$\s*[\d,.]+\s*(R\$\s*[\d,.]+)\s*R\$\s*[\d,.]+'
result_2 = re.findall(regex_2, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

# Regex para obter todas as linhas que possuírem Fundos Imobiliários
regex_3 = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+R\$\s*[\d,.]+[.,]\d{2}\s*(R\$\s*[\d,.]+[.,]\d{2})'
result_3 = re.findall(regex_3, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

# Regex para obter todas as linhas que possuírem Ações
regex_4 = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+R\$\s*[\d,.]+[.,]\d{2}\s*(R\$\s*[\d,.]+[.,]\d{2})'
result_4 = re.findall(regex_4, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

# Preenche o consolidado final
for res in (result_1 + result_2 + result_3 + result_4):

    # Faça atribuições
    ativo = res[0]
    data_vencimento = res[1]
    valor_bruto = res[2]
    
    # Se for encontrada alguma data no nome do ativo, algum registro inválido foi coletado
    if re.search(r'\d{2}\/\d{2}\/\d{4}', ativo):
        continue

    # Caso a data de vencimento encontrada, não seja uma data, altere seu valor para "-"
    if not re.search(r'\d{2}\/\d{2}\/\d{4}', data_vencimento):
        data_vencimento = "-"

    # Adiciona os novos valores ao DataFrame
    df_result = df_result.append({
        'CLASSIFICAÇÃO': obtem_classificacao(res[0]),
        'ATIVO': ativo,
        'EXPOSIÇÃO': "Real",
        'INSTITUIÇÃO': "XP Investimentos",
        'ATUAL': valor_bruto,
        'DATA': data_emissao,
        'ON/OFF': "ON",
        'DATA DE VENCIMENTO': data_vencimento
    }, ignore_index=True)

df_result

,CLASSIFICAÇÃO,ATIVO,EXPOSIÇÃO,INSTITUIÇÃO,ATUAL,DATA,ON/OFF,DATA DE VENCIMENTO
0,Renda Fixa,CDB BANCO C6 S.A.,Real,XP Investimentos,"R$ 90.227,37",07/11/2022,ON,26/11/2025
1,Renda Fixa,LCA BANCO BOCOM BBM SA,Real,XP Investimentos,"R$ 50.967,38",07/11/2022,ON,28/10/2023
2,Renda Fixa,CRA FLORA - J&F,Real,XP Investimentos,"R$ 106.517,20",07/11/2022,ON,15/10/2024
3,Renda Fixa,TREND DI SIMPLES FIRF,Real,XP Investimentos,"R$ 5.328,83",07/11/2022,ON,31/12/2021
4,Ações,PORD11,Real,XP Investimentos,"R$ 153.600,00",07/11/2022,ON,-
5,Ações,BBAS3,Real,XP Investimentos,"R$ 144.250,00",07/11/2022,ON,-
6,Ações,BBDC4,Real,XP Investimentos,"R$ 28.815,00",07/11/2022,ON,-
7,Ações,EZTC3,Real,XP Investimentos,"R$ 122.700,00",07/11/2022,ON,-
8,Ações,HASH11,Real,XP Investimentos,"R$ 19.780,00",07/11/2022,ON,-
9,Ações,LREN3,Real,XP Investimentos,"R$ 147.862,00",07/11/2022,ON,-


In [116]:
result_4

[('PORD11', '11', 'R$ 153.600,00'),
 ('BBAS3', '3', 'R$ 144.250,00'),
 ('BBDC4', '4', 'R$ 28.815,00'),
 ('EZTC3', '3', 'R$ 122.700,00'),
 ('HASH11', '11', 'R$ 19.780,00'),
 ('LREN3', '3', 'R$ 147.862,00'),
 ('LWSA3', '3', 'R$ 92.120,00'),
 ('MGLU3', '3', 'R$ 14.440,00'),
 ('PETR4', '4', 'R$ 569.000,00'),
 ('TRAD3', '3', 'R$ 59.800,00'),
 ('VIIA3', '3', 'R$ 183.750,00')]

In [28]:
# Regex para obter a data de emissão
regex_0 = "(?<=Movimenta..o de \d{2}\/\d{2}\/\d{4} a )\d{2}\/\d{2}\/\d{4}"
data_emissao = re.search(regex_0, text_pdfminer, flags=(re.IGNORECASE)).group(0)

# Regex para obter o texto situado entre o termo "POSIÇÃO CONSOLIDADA" e o termo "Emissão"
regex_1 = "(?<=POSI..O CONSOLIDADA).*?(?=Emissão:)"
result_1 = re.search(regex_1, text_pdfminer, flags=(re.IGNORECASE | re.DOTALL)).group(0)

# Regex para obter todas as linhas que possuírem pelo menos uma letra e um espaço
regex_2 = "^.*[a-zA-Z] .*$"
result_2 = re.finditer(regex_2, result_1, flags=(re.MULTILINE))

# Gera um dicionário de termos com espaço
dictionary = {}
print("TEXTOS A SEREM CORRIGIDOS:")
for result in result_2:
    value = result.group(0)
    dictionary[value.replace(" ","")] = value
    print(" - " + value)

TEXTOS A SEREM CORRIGIDOS:
 - Valor Aplicado
 - Valor Bruto
 - SulAmérica Inflatie FI Renda Fixa Longo Prazo
 - XP Referenciado FI Referenciado DI CP
 - XPA MM II FIC FIM
 - XPA RV FIC FIM
 - XPA Vértice Crédito FIM CP


In [29]:
# Regex para obter o texto situado entre o termo "POSIÇÃOCONSOLIDADA" e o termo "TotalnaInstituição"
regex_3 = "(?<=POSI..OCONSOLIDADA\n).*?(?=TotalnaInstituição)"
result_3 = re.search(regex_3, text_pypdf, flags=(re.IGNORECASE | re.DOTALL)).group(0)

# Substitui os espaços por um ponto e vírgula, para simular uma tabela CSV
result_3 = result_3.replace(" ",";")
print(result_3)

Fundo;Cota;Quantidade;ValorAplicado;ValorBruto;IR;IOF;ValorLiquido
SulAméricaInflatieFIRendaFixaLongoPrazo;49,03907260;485,46020696;21.434,59;23.806,52;13,80;0,00;23.792,72
XPReferenciadoFIReferenciadoDICP;2,55168782;9.607,86926679;23.303,62;24.516,28;272,84;0,00;24.243,44
XPAMMIIFICFIM;1,48495549;206.015,91600648;231.165,15;305.924,47;2.095,63;0,00;303.828,84
XPARVFICFIM;1,51471586;86.244,99799744;119.600,00;130.636,67;1.655,50;0,00;128.981,17
XPAVérticeCréditoFIMCP;1,31704969;62.558,82325043;67.664,32;82.393,08;816,74;0,00;81.576,34



In [30]:
# Altera todos os textos incorretos para a versão correta
for key in dictionary:
    if key in result_3:
        result_3 = result_3.replace(key,dictionary[key])

print(result_3)

Fundo;Cota;Quantidade;Valor Aplicado;Valor Bruto;IR;IOF;ValorLiquido
SulAmérica Inflatie FI Renda Fixa Longo Prazo;49,03907260;485,46020696;21.434,59;23.806,52;13,80;0,00;23.792,72
XP Referenciado FI Referenciado DI CP;2,55168782;9.607,86926679;23.303,62;24.516,28;272,84;0,00;24.243,44
XPA MM II FIC FIM;1,48495549;206.015,91600648;231.165,15;305.924,47;2.095,63;0,00;303.828,84
XPA RV FIC FIM;1,51471586;86.244,99799744;119.600,00;130.636,67;1.655,50;0,00;128.981,17
XPA Vértice Crédito FIM CP;1,31704969;62.558,82325043;67.664,32;82.393,08;816,74;0,00;81.576,34



In [31]:
# Converte o CSV para o tipo DataFrame
df = pd.read_csv(StringIO(result_3), sep=";")
df

,Fundo,Cota,Quantidade,Valor Aplicado,Valor Bruto,IR,IOF,ValorLiquido
0,SulAmérica Inflatie FI Renda Fixa Longo Prazo,"49,03907260","485,46020696","21.434,59","23.806,52","13,80","0,00","23.792,72"
1,XP Referenciado FI Referenciado DI CP,"2,55168782","9.607,86926679","23.303,62","24.516,28","272,84","0,00","24.243,44"
2,XPA MM II FIC FIM,"1,48495549","206.015,91600648","231.165,15","305.924,47","2.095,63","0,00","303.828,84"
3,XPA RV FIC FIM,"1,51471586","86.244,99799744","119.600,00","130.636,67","1.655,50","0,00","128.981,17"
4,XPA Vértice Crédito FIM CP,"1,31704969","62.558,82325043","67.664,32","82.393,08","816,74","0,00","81.576,34"


In [32]:
# Preenche o consolidado final
for index, row in df.iterrows():

    df_result = df_result.append({
        'CLASSIFICAÇÃO': obtem_classificacao(row['Fundo']),
        'ATIVO': row['Fundo'],
        'EXPOSIÇÃO': "Real",
        'INSTITUIÇÃO': "XP Investimentos",
        'ATUAL': row["Valor Bruto"],
        'DATA': data_emissao,
        'ON/OFF': "ON",
        'DATA DE VENCIMENTO': "-"
    }, ignore_index=True)

df_result

,CLASSIFICAÇÃO,ATIVO,EXPOSIÇÃO,INSTITUIÇÃO,ATUAL,DATA,ON/OFF,DATA DE VENCIMENTO
0,Renda Fixa,SulAmérica Inflatie FI Renda Fixa Longo Prazo,Real,XP Investimentos,"23.806,52",11/11/2022,ON,-
1,Renda Fixa,XP Referenciado FI Referenciado DI CP,Real,XP Investimentos,"24.516,28",11/11/2022,ON,-
2,Multimercado,XPA MM II FIC FIM,Real,XP Investimentos,"305.924,47",11/11/2022,ON,-
3,Multimercado,XPA RV FIC FIM,Real,XP Investimentos,"130.636,67",11/11/2022,ON,-
4,Renda Fixa / Multimercado,XPA Vértice Crédito FIM CP,Real,XP Investimentos,"82.393,08",11/11/2022,ON,-


In [33]:
print("Disponibilizando arquivo de consolidado...")

for file in file_objects.keys():

    file_name, file_extension = os.path.splitext(file)

    df_result.to_excel(file_name + ".xlsx", index=False) 
    files.download(file_name + ".xlsx")

Disponibilizando arquivo de consolidado...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>